# Stock Monitor

## Overview

## Import Libraries

In [265]:
import pandas as pd
import numpy as np
import yfinance as yf
from pyfinviz import Screener
from pyfinviz import Insider
import re
import seaborn as sns

# Stock Filter

In [266]:
# stock filter 1: high dividen stocks in US
filter1 = [
  Screener.CountryOption.USA,
  Screener.DividendYieldOption.OVER_10_PERCENT
]

In [267]:
# get max page details from finviz screener 
page_start = 900
pages = list(range(900, 901))
screener_obj = Screener(
  filter_options = filter1,
  pages = pages
)
page_end = int(np.ceil(int(screener_obj.data_frames[page_start].No[0]) / 20))

In [268]:
# load correct finviz screener object given updated max page
page_start = 1
pages = list(range(page_start, page_end))
screener_obj = Screener(
  filter_options = filter1,
  pages = pages
)

In [269]:
# initialize list of screener table results
df_screener_list = []

# build list of screener table results
for i in pages:
  temp1 = screener_obj.data_frames[i]
  df_screener_list.append(temp1)

# join list of screener table results together
df_screener = pd.concat(df_screener_list).reset_index()

df_screener

,index,No,Ticker,Company,Sector,Industry,Country,MarketCap,PE,Price,Change,Volume
0,0,1,AAPB,GraniteShares 2x Long AAPL Daily ETF,Financial,Exchange Traded Fund,USA,-,-,17.76,2.66%,"192,654"
1,1,2,ABR,Arbor Realty Trust Inc.,Real Estate,REIT - Mortgage,USA,2.42B,7.32,12.84,-0.54%,"2,417,783"
2,2,3,ACP,Abrdn Income Credit Strategies Fund,Financial,Closed-End Fund - Debt,USA,359.04M,6.62,6.89,0.73%,"266,064"
3,3,4,ACRE,Ares Commercial Real Estate Corp,Real Estate,REIT - Mortgage,USA,409.51M,-,7.53,1.28%,"350,853"
4,4,5,AFCG,AFC Gamma Inc,Real Estate,REIT - Specialty,USA,242.66M,11.70,11.86,2.15%,"91,904"
...,...,...,...,...,...,...,...,...,...,...,...,...
255,15,256,TSLY,YieldMax TSLA Option Income Strategy ETF,Financial,Exchange Traded Fund,USA,-,-,15.97,-1.12%,"1,762,450"
256,16,257,TUGN,STF Tactical Growth & Income ETF,Financial,Exchange Traded Fund,USA,-,-,23.59,-1.30%,"4,140"
257,17,258,TWO,Two Harbors Investment Corp,Real Estate,REIT - Mortgage,USA,1.33B,-,12.84,0.23%,"797,775"
258,18,259,TYLG,Global X Information Technology Covered Call &...,Financial,Exchange Traded Fund,USA,-,-,31.90,-0.57%,"6,239"


In [270]:
# get screeer tickers
ticker_list = df_screener.Ticker
ticker_list

0      AAPB
1       ABR
2       ACP
3      ACRE
4      AFCG
       ... 
255    TSLY
256    TUGN
257     TWO
258    TYLG
259     UAN
Name: Ticker, Length: 260, dtype: object

## Stock Monitor

In [271]:
# user define function to return filtered top n stocks
def jra_stock_monitor(filter, order_by, order_direction, top_n):
  # get max page details from finviz screener 
  page_start = 900
  pages = list(range(900, 901))
  screener_obj = Screener(
    filter_options = filter,
    pages = pages,
  )
  page_end = int(np.ceil(int(screener_obj.data_frames[page_start].No[0]) / 20))

  # load correct finviz screener object given updated max page
  page_start = 1
  pages = list(range(page_start, page_end))
  screener_obj = Screener(
    filter_options = filter,
    pages = pages,
    order_by = order_by, 
    order_direction = order_direction
  )

  # initialize list of screener table results
  df_screener_list = []

  # build list of screener table results
  for i in pages:
    temp1 = screener_obj.data_frames[i]
    df_screener_list.append(temp1)

  # join list of screener table results together
  df_screener = pd.concat(df_screener_list).reset_index(drop = 'index')

  # show top 10 by price change
  df_screener = df_screener[0:top_n]
  
  return(df_screener)

In [ ]:
# define filter
filter = [
    Screener.CountryOption.USA,
    Screener.DividendYieldOption.OVER_10_PERCENT
  ]

In [ ]:
# define order by criteria
order_by = Screener.OrderBy.CHANGE

# define order direction
order_direction = Screener.OrderDirection.ASC

# define top number of  stocks to return
top_n = 20

In [272]:
# run stock monitor
df_stock_monitor = jra_stock_monitor(
  filter = filter, 
  order_by = order_by, 
  order_direction = order_direction, 
  top_n = top_n
)
df_stock_monitor

,No,Ticker,Company,Sector,Industry,Country,MarketCap,PE,Price,Change,Volume
0,1,RWAY,Runway Growth Finance Corp,Financial,Credit Services,USA,468.30M,10.56,11.56,-14.37%,"2,473,250"
1,2,DKL,Delek Logistics Partners LP,Energy,Oil & Gas Refining & Marketing,USA,1.71B,13.53,39.19,-12.04%,"700,412"
2,3,CHMI,Cherry Hill Mortgage Investment Corporation,Real Estate,REIT - Mortgage,USA,89.03M,-,3.30,-10.33%,"915,376"
3,4,IEP,Icahn Enterprises L P,Energy,Oil & Gas Refining & Marketing,USA,7.76B,-,18.09,-8.27%,"2,114,781"
4,5,EVC,Entravision Communications Corp.,Communication Services,Advertising Agencies,USA,132.01M,-,1.50,-5.65%,"3,792,955"
5,6,MPW,Medical Properties Trust Inc,Real Estate,REIT - Healthcare Facilities,USA,2.53B,-,4.22,-5.07%,"13,212,447"
6,7,NVDY,YieldMax NVDA Option Income Strategy ETF,Financial,Exchange Traded Fund,USA,-,-,27.79,-4.44%,"1,486,686"
7,8,NRT,North European Oil Royalty Trust,Energy,Oil & Gas E&P,USA,48.76M,2.30,5.30,-3.72%,"59,899"
8,9,BGFV,Big 5 Sporting Goods Corp,Consumer Cyclical,Specialty Retail,USA,88.97M,-,3.96,-2.82%,"326,067"
9,10,FBL,GraniteShares 2x Long META Daily ETF,Financial,Exchange Traded Fund,USA,-,-,143.26,-2.75%,"118,596"


In [ ]:
# stock attributes (load directly from github attribute list)
filename1 = 'https://github.com/notfakearcher/julian/raw/main/02_data/stock_term_descriptions.xlsx'
pd.read_excel(io = filename1, header = 0, sheet_name = 'stock_attributes', engine = 'openpyxl')

In [291]:




attributes = pd.DataFrame(yf.Tickers('DKL').tickers['DKL'].info).columns.to_list()

# stock ticker list
tickers = df_stock_monitor.Ticker

# ticker list as joined one-long-single-spaced string
tickers_str = " ".join(tickers)

# generate ticker object
ticker_obj = yf.Tickers(tickers_str)

# initialize stock metrics dataframe
df_stock_metrics = pd.DataFrame(columns = attributes)

# get ticker information based on attributes


In [292]:
df_stock_metrics

,address1,address2,city,state,zip,country,phone,website,industry,industryKey,...,returnOnAssets,freeCashflow,operatingCashflow,earningsGrowth,revenueGrowth,grossMargins,ebitdaMargins,operatingMargins,financialCurrency,trailingPegRatio


In [275]:
tickers0 = df_stock_monitor.Ticker
tickers1 = " ".join(tickers0)
ticker_obj = yf.Tickers(tickers1)
# ticker_obj.tickers[tickers0[0]].info['currentPrice']
result1 = []
result2 = []
result3 = []
result4 = []
result5 = []
for x in tickers0:
  try:
    temp1 = ticker_obj.tickers[x].info['symbol']
    temp2 = ticker_obj.tickers[x].info['previousClose']
    temp3 = ticker_obj.tickers[x].info['currentPrice']
    temp4 = np.round((temp3 - temp2) / temp2 * 100, 2)
    temp5 = ticker_obj.tickers[x].info['volume']
    result1.append(temp1)
    result2.append(temp2)
    result3.append(temp3)
    result4.append(temp4)
    result5.append(temp5)
  except:
    pass
    
df_monitor = pd.DataFrame({
  'ticker': result1,
  'previousClose': result2,
  'currentPrice': result3,
  'percentChange': result4,
  'volume': result5
})
df_monitor = df_monitor.sort_values(by = 'percentChange', ascending = False)
df_monitor

,ticker,previousClose,currentPrice,percentChange,volume
8,HTY,5.0900,5.1300,0.79,29182
13,BRW,7.3550,7.3289,-0.35,157144
12,MSDL,20.3600,20.1900,-0.83,26191
11,PFN,7.4082,7.3450,-0.85,408207
9,GECC,10.8000,10.7000,-0.93,9055
14,OUT,16.1100,15.9350,-1.09,1328056
15,PRT,4.4500,4.3801,-1.57,62983
7,BGFV,4.0800,3.9800,-2.45,371488
10,DALN,4.0200,3.9131,-2.66,9765
3,IEP,18.7200,18.1323,-3.14,2160503


In [276]:
tickers.tickers['DKL'].info

{'address1': '310 Seven Springs Way',
 'address2': 'Suite 400 and 500',
 'city': 'Brentwood',
 'state': 'TN',
 'zip': '37027',
 'country': 'United States',
 'phone': '615 771 6701',
 'website': 'https://www.deleklogistics.com',
 'industry': 'Oil & Gas Refining & Marketing',
 'industryKey': 'oil-gas-refining-marketing',
 'industryDisp': 'Oil & Gas Refining & Marketing',
 'sector': 'Energy',
 'sectorKey': 'energy',
 'sectorDisp': 'Energy',
 'longBusinessSummary': 'Delek Logistics Partners, LP provides gathering, pipeline, transportation, and other services for crude oil, intermediates, refined products, natural gas, storage, wholesale marketing, terminalling water disposal and recycling customers in the United States. The Gathering and Processing segment consists of pipelines, tanks, and offloading facilities that provide crude oil and natural gas gathering and processing, water disposal and recycling, and storage services, as well as crude oil transportation services to third parties. T